In [ ]:
import sys
from pathlib import Path

# 현재 폴더: ptta/other_method/ActMAD/
# ptta 바로 위의 디렉토리를 sys.path에 추가
PROJECT_PARENT = Path.cwd().parents[0]  # -> ptta/ 의 부모 디렉토리
print(PROJECT_PARENT)
sys.path.insert(0, str(PROJECT_PARENT))

from os import path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from ttadapters.datasets import BaseDataset, DatasetHolder, DataLoaderHolder
from ttadapters.datasets import SHIFTClearDatasetForObjectDetection, SHIFTCorruptedDatasetForObjectDetection, SHIFTDiscreteSubsetForObjectDetection

from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator, notebook_launcher

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

# import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

## Check GPU Availability

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 6
ADDITIONAL_GPU = 0
DATA_TYPE = torch.bfloat16

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

In [ ]:
PROJECT_NAME = "ActMAD test"
RUN_NAME = "RT-DETR_R50_ActMAD"

## Baseline

In [ ]:
import baseline
data_root = path.normpath(path.join(Path.cwd(), "..", "data"))
print(data_root)
direct_method = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./direct_method_checkpoints",
    method="direct_method",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=32,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

actmad = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./actmad_checkpoints",
    method="actmad",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

norm = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./norm_checkpoints",
    method="norm",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

dua = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./dua_checkpoints",
    method="dua",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

In [ ]:
direct_method.train()

In [ ]:
actmad.train()

In [ ]:
norm.train()

In [ ]:
dua.train()